<a href="https://www.kaggle.com/code/shravankumar147/01-nanogpt?scriptVersionId=186347566" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [15]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-06-30 16:16:26--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt'

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-06-30 16:16:26 (24.6 MB/s) - 'input.txt' saved [1115394/1115394]



In [16]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [17]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [18]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [19]:
# create a mapping from characters to integers

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [20]:
print(encode("hello"))

[46, 43, 50, 50, 53]


In [21]:
print(decode(encode("hello")))

hello


In [22]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch

print(f"No. of chars: {len(text)}")
print(f"No. of encoded integers: {len(encode(text))}") # => 1115394; 
assert len(text)==len(encode(text)) #this confirms our ecoder is working fine


No. of chars: 1115394
No. of encoded integers: 1115394


In [23]:
encoded_text = encode(text)
data = torch.tensor(encoded_text, dtype=torch.long)

print(data.shape, data.dtype)

print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [24]:
# Let's now split up the data into train and validation sets
# first 90% will be train, rest val
print(f"Data Size: {len(data)}")
n = int(len(data)*0.9)

print(f"90%: {len(data[:n])}| 10%: {len(data[n:])}")

print(f"90%: {len(data[:n])} + 10%: {len(data[n:])} =  {len(data[:n]) + len(data[n:])}")
assert len(data[:n]) + len(data[n:]) == len(data)


train_data = data[:n]
val_data = data[n:]

Data Size: 1115394
90%: 1003854| 10%: 111540
90%: 1003854 + 10%: 111540 =  1115394


In [25]:
block_size = 8 # also known as context length, that a transformer see

train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [26]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

print(f"input context-> target")
print("="*50)

for t in range(block_size):    
    print(f"{x[:t+1]} -> {y[t]}")

input context-> target
tensor([18]) -> 47
tensor([18, 47]) -> 56
tensor([18, 47, 56]) -> 57
tensor([18, 47, 56, 57]) -> 58
tensor([18, 47, 56, 57, 58]) -> 1
tensor([18, 47, 56, 57, 58,  1]) -> 15
tensor([18, 47, 56, 57, 58,  1, 15]) -> 47
tensor([18, 47, 56, 57, 58,  1, 15, 47]) -> 58


In [27]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    data = train_data if split=='train' else val_data
    
    # This line generates batch_size random indices (ix) within the range from 0 to len(data) - block_size. 
    # The subtraction by block_size ensures there’s enough room to create sequences of length block_size.
    ix = torch.randint(low=0, high=len(data)-block_size, size=(batch_size,)) # we get batch_size random integers within the range
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    return x,y

In [28]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


In [29]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b, t]
        
        print(f"when the input is: {context.tolist()}, the target is: {target}")

when the input is: [24], the target is: 43
when the input is: [24, 43], the target is: 58
when the input is: [24, 43, 58], the target is: 5
when the input is: [24, 43, 58, 5], the target is: 57
when the input is: [24, 43, 58, 5, 57], the target is: 1
when the input is: [24, 43, 58, 5, 57, 1], the target is: 46
when the input is: [24, 43, 58, 5, 57, 1, 46], the target is: 43
when the input is: [24, 43, 58, 5, 57, 1, 46, 43], the target is: 39
when the input is: [44], the target is: 53
when the input is: [44, 53], the target is: 56
when the input is: [44, 53, 56], the target is: 1
when the input is: [44, 53, 56, 1], the target is: 58
when the input is: [44, 53, 56, 1, 58], the target is: 46
when the input is: [44, 53, 56, 1, 58, 46], the target is: 39
when the input is: [44, 53, 56, 1, 58, 46, 39], the target is: 58
when the input is: [44, 53, 56, 1, 58, 46, 39, 58], the target is: 1
when the input is: [52], the target is: 58
when the input is: [52, 58], the target is: 1
when the input i

In [30]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [31]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [32]:
emb = nn.Embedding(vocab_size, vocab_size)

Here we test the Embedding layer

It accepts a tensor as input

In [33]:
print(emb(torch.tensor(0))) # for a 1-d tensor it has generated below output
print(emb(torch.tensor(0)).shape) # the shape is 65, which are embeddings from the embedding lookup table

tensor([ 0.1808, -0.0700, -0.3596, -0.9152,  0.6258,  0.0255,  0.9545,  0.0643,
         0.3612,  1.1679, -1.3499, -0.5102,  0.2360, -0.2398, -0.9211,  1.5433,
         1.3488, -0.1396,  0.2858,  0.9651, -2.0371,  0.4931,  1.4870,  0.5910,
         0.1260, -1.5627, -1.1601, -0.3348,  0.4478, -0.8016,  1.5236,  2.5086,
        -0.6631, -0.2513,  1.0101,  0.1215,  0.1584,  1.1340, -1.1539, -0.2984,
        -0.5075, -0.9239,  0.5467, -1.4948, -1.2057,  0.5718, -0.5974, -0.6937,
         1.6455, -0.8030,  1.3514, -0.2759, -1.5108,  2.1048,  2.7630, -1.7465,
         1.4516, -1.5103,  0.8212, -0.2115,  0.7789,  1.5333,  1.6097, -0.4032,
        -0.8345], grad_fn=<EmbeddingBackward0>)
torch.Size([65])


We can pass a batch of input to the embedding layer, so let's pass our xb batch

In [34]:
xb.shape

torch.Size([4, 8])

In [35]:
logits = emb(xb)
B,T,C = logits.shape
print(B,T,C)

4 8 65


4 is the batch size

8 is the block size (time-dimention)

65 is the output embedding (channel -dimention)

Let's do some reshaping - this will come handy when we compute cross-entropy loss for our Bigram model

In [36]:
logits.view(B*T, C).shape # using view approach

torch.Size([32, 65])

In [37]:
logits.flatten(start_dim=0, end_dim=1).shape # using flatten approach

torch.Size([32, 65])

We can use any method, so I am chosing view, because I can use my pre computed B,T, and C there

In [38]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is None:
            loss = None
        else:
            
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
        
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx) # Logits has shape: (B, T, C)
            
            # Let's take the last time step from the logits/predictions
            logits = logits[:,-1,:] #(B, C)
            
            # apply softmax to get the probabilities of these predictions
            # over the Channels dimensions. 
            probs = F.softmax(logits, dim=-1) #(B, C)
            
            # torch.multinomial returns a tensor where each row contains num_samples indices 
            # sampled from the multinomial probability distribution located in the corresponding 
            # row of tensor input.
            
            idx_next = torch.multinomial(probs, num_samples=1)
            
            # add the next probable prediction to the current idx batch.
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            
        return idx   
        

In [39]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.6630, grad_fn=<NllLossBackward0>)


In [40]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


hbH

:CLP.A!fq'3ggt!O!T?X!!SA?W&TrpvYybSE3w&S BXUhmiKYyTmWMPhhmnHKj!!btgnwNNULuEzRuYyiWEQxPX!$3C'MBj


That's something so random, that means we need to learn to make good predictions

In [41]:
# create a pytorch optiizer 
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [42]:
# training loop
# get batches -> compute loss -> get gradients from the computed loss -> updated model params 

batch_size = 32
num_epochs = 10000

for steps in range(num_epochs):
    
    #sample the batches from the data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if steps%1000==0:
        print(f"step: {steps} - {loss.item()}")
        
print(f"\nfinal loss at end of epoch {steps}: {loss.item()}")

step: 0 - 4.641923427581787
step: 1000 - 3.615529775619507
step: 2000 - 3.0686445236206055
step: 3000 - 2.7126073837280273
step: 4000 - 2.6058950424194336
step: 5000 - 2.6941914558410645
step: 6000 - 2.480062246322632
step: 7000 - 2.421450614929199
step: 8000 - 2.602724075317383
step: 9000 - 2.419046640396118

final loss at end of epoch 9999: 2.451293468475342


In [43]:
idx = torch.zeros((1, 1), dtype=torch.long)
idx_new = m.generate(idx, max_new_tokens=500)

print(decode(idx_new[0].tolist()))


CI
TEE:
NClucor che t thendouilftheesco'imin?
DUESp;
Dupll nn owe hisaloriclfer eened de ms, sth:
MI dintearow, att t isin aren gs fran bag ndais theer so,

IVI mounce s pr higay'stindinthe,

Why w'surnurn myof bby fedise?
Thertle:
LOLO, e te.
FzirJond f:
S: winan w, KI ardathart.

D m arere toor!
Argo sh meas hirend


As anicrirtopowh ovend sinouce doreaklanoreseranmye, bithorer glre?
ARS: t i'CLown athe is! hinondar ate t y poames.
Thathim IR.
OTryowifromy te pehothitowild. hoveaswecestharchel


## The mathematical trick in self-attention

In [44]:
import torch
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [59]:
# consider the following toy example:

torch.manual_seed(1338)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [61]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [70]:
print(xprev.shape)
torch.mean(xprev, 0)
# xbow[0,2]

torch.Size([8, 2])


tensor([-0.0373, -0.6127])

In [73]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
print(wei.shape, x.shape)

xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)  i.e (T,T) X (T,C) --> (T,C) with batch B
# xbow2
torch.allclose(xbow, xbow2)

torch.Size([8, 8]) torch.Size([4, 8, 2])


True

In [75]:
# # Step 1: Add a new dimension at position 0
# wei = wei.unsqueeze(0)  # Shape becomes [1, 8, 8]

# # Step 2: Repeat the tensor along the new dimension
# wei = wei.repeat(4, 1, 1)  # Shape becomes [4, 8, 8]

# print(wei.shape)  # Output: torch.Size([4, 8, 8])

torch.Size([4, 8, 8])


In [78]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [33]:
# version 4: self- attention

import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1337)

B, T, C = 4, 8, 32
head_size = 16

x = torch.randn(B,T,C)

key   = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False) 
value = nn.Linear(C, head_size, bias=False) 


k = key(x)  # (B,T,C)(C,head_size) -> (B,T,head_size)
q = query(x) # (B,T,C)(C,head_size) -> (B,T,head_size)



wei = q@k.transpose(-2,-1)  # (B,T,head_size) @ (B,head_size,T) --> (B, T, T)

# Decoder , we don't want the communication happen from the future tokens in the attention mechanism
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0, float("-inf")) # (B, T, T)

wei = F.softmax(wei, dim=-1)

v = value(x)

out = wei@v  # (B,T,T) @ (B,T,C)  --> (B,T,C)

print(out.shape)
 


torch.Size([4, 8, 16])


In [34]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [35]:
out[0].shape

torch.Size([8, 16])

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below